In [ ]:
## IMPORT MODULES ##

from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
from Functions import *

In [ ]:
## MONGO CONECTION ##

client = MongoClient("mongodb://localhost:27017/")
db = client.companies 

In [ ]:
## MONGO QUERY ##

filters = {"$and":[
{"offices": {"$exists": True}},{"offices": {"$ne": None}}, 
{"category_code": {"$exists": True}},{"category_code": {"$ne": None}},
{"founded_year": {"$exists": True}}, {"founded_year": {"$gt": 2009}},
{"total_money_raised": {"$exists": True}},{"total_money_raised":{"$ne":None}},
{"total_money_raised": {"$not":{"$size":0}}}, 
{"$or": [
    {"total_money_raised": {"$gte": 1_000_000}},
    {"category_code": "design" } ,
    {"category_code": "web" } , 
    {"category_code": "software" } , 
    {"category_code": "games_video" } , 
    {"category_code": "mobile" } , 
    {"category_code": "enterprise" } ,   
    {"category_code": "analytics" } ,
    {'category_code': "search"},
    {'category_code': "network_hosting"} ,   
    {"category_code": "photo_video"}]}]}

projection = {"_id": 0, "crunchbase_url": 0, "products": 0, "acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
"screenshots": 0, "external_links": 0, "partners": 0, "image": 0, "funding_rounds": 0, "updated_at": 0, 
"deadpooled_year":0, "deadpooled_month": 0, "deadpooled_day":0, "deadpooled_url": 0, "number_of_employees": 0,
"providerships":0, "blog_url": 0, "blog_feed_url": 0, "tag_list":0, "alias_list":0, "relationships":0,
"competitions":0,"milestones":0, "founded_month":0, "founded_day":0, "twitter_username":0, "homepage_url":0, 
 "overview":0,"phone_number":0,"investments":0,"permalink":0,"email_address":0, "description":0,"created_at":0}


In [ ]:
## WE CREATE THE PANDAS DF ##

companies = db.companies.find(filters,projection)
data_companies = pd.DataFrame(companies)

In [ ]:
## WE UNPACK EVERY COMPANY´S OFFICES ##


unpack = [ { '$unwind' : '$offices' },{ '$project': {"_id":0} }, {"$out":"companies"}]
companies.collection.aggregate(unpack)

offices_df = json_normalize(data_companies["offices"])


In [ ]:
## CONCAT BOTH DF INTO MAIN COMPANY DF ##

full_df = pd.concat([data_companies,offices_df],axis=1)
#full_df.head()


In [ ]:
## CREATE GEOPOINT COLUMN FOR MONGODB ##

full_df["geopoint"] = full_df.apply(lambda x: geopoint(x["longitude"],x["latitude"]),axis = 1)

In [ ]:
## WE FILTER THE DF BY DELETING NULL AND DUPLICATE ROWS ##

full_clean_df = full_clean_df.dropna(subset=['latitude'])
full_clean_df.drop_duplicates
display(full_clean_df.head())


In [ ]:
## FILL MISSING CITY VALUE ##

full_clean_df.loc[0,"city"] = "Dearing"
full_clean_df.head()


In [ ]:
## SAVE AS JSON ##

full_clean_df.to_json("Filtered_Companies.json", orient = "records", lines = True)